### Вычисление эмбеддингов документов с помощью модели BERT

In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import warnings
from multiprocessing import Process, Lock
import tensorflow_hub as hub
import tensorflow_text
import torch

ModuleNotFoundError: No module named 'pandas'

In [6]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

In [7]:
with open("text_topic_mapping.pkl", "rb") as file_to_read:
    text_topic_mapping = pickle.load(file_to_read)

In [8]:
text_topic_mapping

,file_name,topic
0,1579769179.72238.txt,4
1,1579534063.62363.txt,6
2,1579509983.59048.txt,1
3,1579088437.47050.txt,10
4,1579161966.49942.txt,0
...,...,...
62067,1556196307.242144.txt,5
62068,1556621827.252476.txt,10
62069,1556288216.247176.txt,8
62070,1556283861.246650.txt,8


Считываем данные из файлов в массив строк

In [26]:
TRANSCRIBE_PATH = "/media/storage/alekseychuk/LDA/recognition"

data_strings = []
file_names = []

for folder in os.listdir(TRANSCRIBE_PATH):
    if folder.startswith("."):
        continue
    print("folder: " + folder)
    folder_path = TRANSCRIBE_PATH + '/' + folder
    for text_name in os.listdir(folder_path):
        if text_name.startswith(".") or text_name not in text_topic_mapping['file_name'].values:
            continue
        text_path = folder_path + '/' + text_name
        f = open(text_path, 'r')
        words = f.read()
        if words:
            split = words.split()
            if len(split) > 150:
                data_strings.append(words)
                file_names.append(text_name)
                    


folder: 1579
folder: 1564
folder: 1541
folder: 1604
folder: 1601
folder: 1581
folder: 1573
folder: 1554
folder: 1600
folder: 1597
folder: 1569
folder: 1552
folder: 1585
folder: 1575
folder: 1546
folder: 1588
folder: 1565
folder: 1570
folder: 1598
folder: 1578
folder: 1567
folder: 1584
folder: 1599
folder: 1547
folder: 1592
folder: 1595
folder: 1571
folder: 1596
folder: 1553
folder: 1572
folder: 1568
folder: 1605
folder: 1582
folder: 1566
folder: 1576
folder: 1591
folder: 1590
folder: 1563
folder: 1549
folder: 1555
folder: 1589
folder: 1557
folder: 1545
folder: 1593
folder: 1550
folder: 1543
folder: 1602
folder: 1603
folder: 1583
folder: 1542
folder: 1577
folder: 1548
folder: 1587
folder: 1561
folder: 1544
folder: 1580
folder: 1594
folder: 1586
folder: 1574
folder: 1551
folder: 1562
folder: 1556


In [29]:
len(data_strings)

61200

In [40]:
# Токенизация входных последовательностей (256-мерные векторы)
train_encodings = tokenizer(data_strings, truncation=True, padding='max_length', max_length=256)

In [41]:
input_ids = torch.tensor(train_encodings['input_ids'])

In [42]:
input_ids.shape

torch.Size([61200, 256])

In [43]:
BATCH_LENGTH = 200

In [44]:
# Выходные векторы эмбеддингов имеют размерность 768
embeddings = np.empty((0,768))

In [45]:
device = torch.device("cuda")

In [46]:
device2 = torch.device("cpu")

In [ ]:
# Переводим модель для вычислений на GPU
model.to('cuda')

Вычисляем эмбеддинги по батчам

In [48]:
with torch.no_grad():
    for batch_num in range(int(len(input_ids) / BATCH_LENGTH)):
        try:
            batch = input_ids[batch_num * BATCH_LENGTH : (batch_num + 1) * BATCH_LENGTH].cuda()
            results = model(batch)
            head_output = results[0][:,0,:].cpu().detach().numpy()
            embeddings = np.concatenate((embeddings, head_output))
            print(f"processed batch {batch_num}")
        except Exception:
            pass

        

processed batch 0
processed batch 1
processed batch 2
processed batch 3
processed batch 4
processed batch 5
processed batch 6
processed batch 7
processed batch 8
processed batch 9
processed batch 10
processed batch 11
processed batch 12
processed batch 13
processed batch 14
processed batch 15
processed batch 16
processed batch 17
processed batch 18
processed batch 19
processed batch 20
processed batch 21
processed batch 22
processed batch 23
processed batch 24
processed batch 25
processed batch 26
processed batch 27
processed batch 28
processed batch 29
processed batch 30
processed batch 31
processed batch 32
processed batch 33
processed batch 34
processed batch 35
processed batch 36
processed batch 37
processed batch 38
processed batch 39
processed batch 40
processed batch 41
processed batch 42
processed batch 43
processed batch 44
processed batch 45
processed batch 46
processed batch 47
processed batch 48
processed batch 49
processed batch 50
processed batch 51
processed batch 52
pro

In [49]:
embeddings.shape

(61200, 768)

Сохраняем результаты в файлы

In [50]:
with open('BERT_embeds.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [51]:
with open('embedded_file_names.pkl', 'wb') as f:
    pickle.dump(file_names, f)